<a href="https://colab.research.google.com/github/vipashaaV321/Collaborative-Food-Recipe-Recommendation-System/blob/Vipasha/SVD_Approach_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
Base_path ='/content/drive/MyDrive/Data/RS2'

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
matrix=np.load(Base_path+"/user_item_100.npy")

In [ ]:
# data=data.pivot(index="user_id",columns='recipe_id',values='rating').fillna(0)
U,S,Vt=np.linalg.svd(matrix,full_matrices=False)

In [ ]:
rank=np.count_nonzero(S)

In [ ]:
S=np.diag(S[:rank])
U=U[:,:rank]
Vt=Vt[:rank,:]

In [ ]:
# approximation=np.dot(np.dot(U,S),Vt)+ avg_rating

In [ ]:
# predict the ratings for all uninteracted items for a user
def predict_all(user_id, matrix, item_similarities, k=5):
    # get the ratings of the user
    user_ratings = matrix[user_id, :]

    # get the indices of the uninteracted items
    uninteracted_items = np.where(user_ratings == 0)[0]

    # predict the rating for each uninteracted item
    predictions = np.zeros((len(uninteracted_items),))
    for i, item_id in enumerate(uninteracted_items):
        # find the k most similar items to the target item
        similar_items = np.argsort(-item_similarities[item_id, :])[:k]

        # get the ratings of the k most similar items
        similar_ratings = matrix[user_id, similar_items]

        # compute the weighted average of the ratings
        similarity_scores = item_similarities[item_id, similar_items]
        prediction = np.dot(similarity_scores, similar_ratings) / np.sum(similarity_scores)

        # add the average rating of the item to the prediction
        prediction += np.mean(matrix[:, item_id])

        # store the prediction
        predictions[i] = prediction

    # create a dataframe with the item IDs and predicted ratings
    predicted_ratings = pd.DataFrame({'item_id': uninteracted_items, 'rating': predictions})

    return predicted_ratings

In [ ]:
from scipy.sparse import csr_matrix

# convert the input matrix to a sparse matrix
sparse_matrix = csr_matrix(matrix)

# define the batch size
batch_size = 1000

# compute the similarity matrix between items incrementally
item_similarities = csr_matrix((matrix.shape[1], matrix.shape[1]), dtype=np.float32)
for i in range(0, matrix.shape[1], batch_size):
    print(f"Processing items {i}-{i+batch_size-1}")
    batch = sparse_matrix[:, i:i+batch_size]
    print(batch.shape)
    batch_item_similarities = batch.T.dot(Vt)
    item_similarities[i:i+batch_size, :] = batch_item_similarities

Processing items 0-999
(1165, 1000)


/usr/local/lib/python3.8/dist-packages/scipy/sparse/_index.py:125: SparseEfficiencyWarning: Changing the sparsity structure of a csr_matrix is expensive. lil_matrix is more efficient.
  self._set_arrayXarray(i, j, x)


In [ ]:
print(Vt.T.shape)

(142485, 1165)


###### Getting Ram Crashed try to use chuncked data

In [ ]:
# define the batch size
batch_size = 1000

# compute the similarity matrix between items incrementally
item_similarities = np.zeros((matrix.shape[1], matrix.shape[1]))
for i in range(0, matrix.shape[1], batch_size):
    print(f"Processing items {i}-{i+batch_size-1}")
    batch = matrix[:, i:i+batch_size]
    batch_item_similarities = np.dot(Vt.T[i:i+batch_size, :], Vt.T)
    item_similarities[i:i+batch_size, :] = batch_item_similarities

In [ ]:
item_similarities = np.dot(Vt.T, Vt)